In [1]:
# Import liberaries and functions
import pandas as pd
import glob
import time

from file_preprocess import file_preprocess

In [5]:
# Get all raw html/text file paths
all_files = glob.glob('Data/Risk Factors 10k/*')

In [ ]:
# Create an empty pandas DataFrame
RF_df = pd.DataFrame()
i = 1
start = time.time()

for file in all_files:
    # Read file
    with open(file, 'rb') as f:
        f_name = f.name.split('\\')[-1]
        cik = f_name.split('-')[0]
        r_year = f_name.split('-')[2][1:5]
        f_date = f_name.split('-')[1][1:]
        f_ext = f.name.split('.')[-1]
        
        # Convert the textual data into the required format
        item_1a = file_preprocess(f.read(), format=f_ext)
        
    if item_1a is not None:
        # Add the file and correspomding data to the list in DataFrame
        RF_df = RF_df.append(
            pd.DataFrame(data={'cik': cik, 'reporting year': r_year, 'filing date': f_date, 'Item 1A': item_1a})
        )
    else:
        pass
    
    # Save the DataFrame for every 1000 files
    if (i%500 == 0) or (i == len(all_files)):
        RF_df.to_csv('RF_df.csv')
        print(f'{i} files processed | {time.time()-start}')
            
    i+=1 # Update the counter

In [29]:
# Remove empty reports
RF_df.dropna(inplace=True)

# Remove duplicates
RF_df = (
    RF_df[RF_df['Item 1A'].apply(lambda x: len(x.split()))>=5]
    .sort_values(by=['cik', 'reporting year', 'filing date']).drop_duplicates()
)

# Save the clean DataFrame
RF_df.to_csv('RF_df.csv')